# Seccion A.2
## Datos abiertos de la CDMX

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

alcaldias = [
    'Milpa Alta',
    'Benito Juarez',
    'Gustavo A Madero',
    'Coyoacan',
    'Miguel Hidalgo',
    'La Magdalena Contreras',
    'Tlahuac',
    'Azcapotzalco',
    'Iztacalco',
    'Alvaro Obregon',
    'Xochimilco',
    'Venustiano Carranza',
    'Tlalpan',
    'Cuajimalpa de Morelos',
    'Cuauhtemoc',
    'Iztapalapa'
]
alcaldias = list(map(str.upper, alcaldias))

In [2]:
data = pd.read_csv("carpetas-de-investigacion-pgj-de-la-ciudad-de-mexico.csv")
## We delete all the crimes that do not correspond to cdmx alcaldias
mask_alcaldias_cdmx = data.apply(lambda x: True if x['alcaldia_hechos'] in alcaldias else False, axis=1)
data = data.where(mask_alcaldias_cdmx).dropna(how='all')

In [3]:
data['alcaldia_hechos'].value_counts()

CUAUHTEMOC                170940
IZTAPALAPA                162469
GUSTAVO A MADERO          108085
BENITO JUAREZ              91470
ALVARO OBREGON             72407
COYOACAN                   72039
MIGUEL HIDALGO             70935
TLALPAN                    63076
VENUSTIANO CARRANZA        62448
AZCAPOTZALCO               52272
IZTACALCO                  45181
XOCHIMILCO                 33490
TLAHUAC                    25594
LA MAGDALENA CONTRERAS     16938
CUAJIMALPA DE MORELOS      15634
MILPA ALTA                  7261
Name: alcaldia_hechos, dtype: int64

### Question 1
Which test would you do to ensure the quality of this data?
I would check for the columns unique values, specifically for the ones that are not very numerous, like 'ao_hechos' or the ones that can contain strings, accents tend to cause problems due to its different codification.

If there are error, I would try to correct them and if I can't then I would drop those malformed rows.

### Question 2
How many crimes are registered in the table? What is the time range of these crimes?

In [4]:
registered_crimes = data.shape[0]
print(f'There are {registered_crimes} registered crimes')

last_registered_year = data['ao_hechos'].max()
first_registered_year = data['ao_hechos'].min()
print(f'The time range of this table goes from the years {first_registered_year:.0f} to {last_registered_year:.0f}')


There are 1070239 registered crimes
The time range of this table goes from the years 1906 to 2020


### Question 3
How are the crimes distributed in Mexico city? Which are the 5 most commited crimes?

In [5]:
top_5 = data['delito'].value_counts()[:5]
print(f'The 5 most commited crime are: \n{top_5}')

The 5 most commited crime are: 
VIOLENCIA FAMILIAR                                102671
ROBO A NEGOCIO SIN VIOLENCIA                       64689
ROBO DE OBJETOS                                    64661
FRAUDE                                             62872
ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA     54935
Name: delito, dtype: int64


### Question 4
Identify the crime that are increasing and the ones decreasing compared to the last year (2019)? (be careful with low ocurrency crimes)

In [6]:
wrong_char = ['Ã“', 'Ã', 'Ã‰', 'Ãš', 'Ã\x81']
                
right_char = ['Ó', 'Í', 'É','Ú', 'Á']

#It would be easier to map all the accents to their wrong equivalent i.e. í: Ã, ó:Ã“
data['delito'] = data.apply(lambda row:  right_string[wrong_string.index(row['delito'])] if row['delito'] in wrong_string else row['delito'], axis = 1)


NameError: name 'wrong_string' is not defined

In [ ]:
wrong_string = ['PRODUCCIÃ“N, IMPRESIÃ“N, ENAJENACIÃ“N, DISTRIBUCIÃ“N, ALTERACIÃ“N O FALSIFICACIÃ“N DE TÃTULOS AL PORTADOR, DOCUMENTOS DE CRÃ‰DITO PÃšBLICOS O VALES DE CANJE',
                'USURPACIÃ“N DE IDENTIDAD']
                
right_string = ['PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCIÓN, ALTERACIÓN O FALSIFICACIÓN DE TÍTULOS AL PORTADOR, DOCUMENTOS DE CRÉDITO PÚBLICOS O VALES DE CANJE',
               'USURPACIÓN DE IDENTIDAD']

#It would be easier to map all the accents to their wrong equivalent i.e. í: Ã, ó:Ã“
data['delito'] = data.apply(lambda row:  right_string[wrong_string.index(row['delito'])] if row['delito'] in wrong_string else row['delito'], axis = 1)


In [ ]:
data.head()

In [ ]:
crimes_per_year = data.where(data['ao_hechos']>=2019).value_counts(['delito', 'ao_hechos']).to_frame()
crimes_per_year = crimes_per_year.unstack(fill_value=0)
crimes_per_year.columns = crimes_per_year.columns.droplevel(0)
crimes_per_year.columns.name = None
crimes_per_year.columns = crimes_per_year.columns.astype('int64').astype('string')

# Create the increase column, if negative is a decrease
crimes_per_year['% increase@2020-2019'] = (crimes_per_year['2020']-crimes_per_year['2019'])*100/crimes_per_year['2019']

# We need to set a threshold in the number of crimes
# Remove crimes in 2020 that equal zero
crimes_per_year = crimes_per_year.where(crimes_per_year['2019']>50).dropna()
crimes_per_year = crimes_per_year.where(crimes_per_year['2020']>50).dropna()
crimes_per_year = crimes_per_year.sort_values(by=['% increase@2020-2019'], ascending=False)
crimes_per_year.iloc[:10]

In [ ]:
crimes_per_year.iloc[-10:]

In [ ]:
crimes_per_year = data.where((data['ao_hechos']>=2018) & (data['ao_hechos']<2020)).value_counts(['delito', 'ao_hechos']).to_frame()
crimes_per_year = crimes_per_year.unstack(fill_value=0)
crimes_per_year.columns = crimes_per_year.columns.droplevel(0)
crimes_per_year.columns.name = None
crimes_per_year.columns = crimes_per_year.columns.astype('int64').astype('string')

# Create the increase column, if negative is a decrease
crimes_per_year['% increase@2019-2018'] = (crimes_per_year['2019']-crimes_per_year['2018'])*100/crimes_per_year['2018']

# We need to set a threshold in the number of crimes
# Remove crimes in 2020 that equal zero
crimes_per_year = crimes_per_year.where(crimes_per_year['2018']>50).dropna()
crimes_per_year = crimes_per_year.where(crimes_per_year['2019']>50).dropna()
crimes_per_year = crimes_per_year.sort_values(by=['% increase@2019-2018'], ascending=False)
crimes_per_year.iloc[:10]

In [ ]:
crimes_per_year.iloc[-10:]

### Question 5
Which "alcaldia" has the most crimes and which one the least? Why do you think is this?

In [ ]:
data['alcaldia_hechos'].value_counts()

### Question 6
Inside each "alcaldia", which are the three neighborhoods with the most crimes?

In [ ]:
crimes_per_neighborhood = data.value_counts(['alcaldia_hechos', 'colonia_hechos']).to_frame()

In [ ]:
crimes_per_neighborhood.columns = ['count']

In [ ]:
crimes_per_neighborhood = crimes_per_neighborhood.sort_values(['alcaldia_hechos', 'count'], ascending=[True, False])

In [ ]:
for alcaldia in alcaldias:
    print(f'The three neighborhoods with the most crimes in {alcaldia} are :\n {crimes_per_neighborhood.loc[alcaldia].head(3)}\n')

### Question 7
Is there any seasonal tendency in crimes? (monthly, weekly, biweekly, day of the week)

In [ ]:
seasonal_tendency = data.copy()

#We need to drop the dates that does not have a valid timestamp
def into_timestamp(row):
    try:
        return pd.Timestamp(row).date()
    
    except:
        return np.NaN
    
seasonal_tendency['fecha_hechos'] = seasonal_tendency['fecha_hechos'].apply(lambda row: into_timestamp(row)) # This may not be the cleanest solution
seasonal_tendency = seasonal_tendency.dropna(subset=['fecha_hechos'])
seasonal_tendency['fecha_hechos'] = pd.to_datetime(seasonal_tendency['fecha_hechos'])

In [ ]:
seasonal_tendency['month'] = seasonal_tendency['fecha_hechos'].dt.month
seasonal_tendency['week_day'] = seasonal_tendency['fecha_hechos'].dt.weekday
seasonal_tendency['weekly'] = seasonal_tendency['fecha_hechos'].dt.week
seasonal_tendency['biweekly'] = seasonal_tendency['weekly']//2 + (seasonal_tendency['weekly'] % 2 > 0)


In [ ]:
seasonal_tendency.value_counts('week_day').sort_index().plot(kind='bar')

In [ ]:
seasonal_tendency.value_counts('month').sort_index().plot(kind='bar')

In [ ]:
seasonal_tendency.value_counts('weekly').sort_index().plot(kind='bar')

In [ ]:
seasonal_tendency.value_counts('biweekly').sort_index().plot(kind='bar')

### In a first analysis (visual analysis of plots) it seems as if the crime tends to decrease in the last weeks and months of the year. And also on sundays. It also tends to increase on fridays. Further analysis would be needed to determine if this is significant.

### Question 8
Which are the crimes that characterize each 'alcaldia'?

In [ ]:
crimes_per_alcaldia = data.value_counts(['delito','alcaldia_hechos']).to_frame()
crimes_per_alcaldia.columns = ['count']
crimes_per_alcaldia = crimes_per_alcaldia.groupby('delito').head(1).reset_index()

In [ ]:
top_exclusive_crime = []
for alcaldia in alcaldias:
    try:
        aux = crimes_per_alcaldia.where(crimes_per_alcaldia['alcaldia_hechos'] == alcaldia).dropna().iloc[0].to_list()
        top_exclusive_crime.append(aux)
    except:
        pass

In [ ]:
pd.DataFrame(top_exclusive_crime, columns=crimes_per_alcaldia.columns)

### Question 9
Calcula el número de homicidios dolosos por cada 100 mil habitantes anual para cada Área Geoestadística Básica (AGEB) del INEGI. (hint: no importa que el dato de población no esté actualizado).
a) Pinta un mapa con este indicador. Describe los resultados.

### Question 10
¿Cómo diseñarías un indicador que midiera el nivel “inseguridad”? Diséñalo al nivel de
desagregación que te parezca más adecuado (ej. manzana, calle, AGEB, etc.).

En mi opinion lo realizaria a nivel de colonia. Podria hacerse un conteo de los delitos en cada colonia, dandole un peso diferente a cada uno de ellos. Homicidios, violaciones y secuestros tendrian un mayor peso que el resto de las categorias de delitos. Este puntaje se escalaria a un rango de cero a diez, en donde cero es el minimo y diez el maximo. Cada colonia tendria un puntaje con base en esto. 





### Question 11
Con alguna de las medidas de crimen que calculaste en los incisos anteriores, encuentra
patrones de concentración geográfica de delitos (hint: puedes usar algoritmos de
Machine Learning no supervisados).
a) ¿Qué caracteriza a cada punto de concentración de delitos y qué tienen en
común?

### Question 12
Toma los delitos clasificados como “Robo a pasajero a bordo de transporte público con
y sin violencia”. ¿Cuáles son las ruta de transporte público donde más ocurren estos
delitos?

In [ ]:
crimes_list = list(data['delito'].unique())
public_transport_crimes_list = [row  for row in crimes_list if ('ROBO A PASAJERO' in row) and ('CONDUCTOR' not in row)]

In [ ]:
list(data['categoria_delito'].unique())

In [ ]:
# Keep only the crimes that involve public transport
mask_transport_crimes = data.apply(lambda x: True if x['delito'] in public_transport_crimes_list else False, axis=1)
public_transport_df = data.where(mask_transport_crimes).dropna(how='all')


In [ ]:
public_transport_df.value_counts(['calle_hechos']).to_frame().head(10)